# Script for crafting Adversarial Examples

In [2]:
import os

import utils.text_processing as tp

In [3]:
# get data from important word detector
%store -r important_words
%store -r sentence_packages

## ;Method, 3: ?punctuation

Plan:
im ersten Schritt: 
nach dem 
important Word , einfügen

#### 1. Create modified Dataset

In [4]:
original_sentences = []
for sentence in sentence_packages:
    original_sentences.append(sentence['original_sentence'])

In [5]:
modified_words = []
for word in important_words:
    modified_words.append(tp.to_punctuation(word))

In [6]:
modified_sentences = tp.generate_modified_sentences(original_sentences, important_words, modified_words)

#### 2. Import BERT Model

In [9]:
import torch
import random
import numpy as np
import pandas as pd

from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model.eval();

#### 3. Prediction

In [10]:
original_predictions = []
for sentence in original_sentences:
    original_predictions.append(tp.predict_sentiment(model, tokenizer, sentence))

In [11]:
modified_predictions = []
for sentence in modified_sentences:
    modified_predictions.append(tp.predict_sentiment(model, tokenizer, sentence))

#### 4. Creation of Dataset

In [12]:
ds = pd.DataFrame(list(zip(original_sentences, original_predictions, modified_sentences, modified_predictions)),
                 columns = ['original_sentences', 'original_predictions', 'modified_sentences', 'modified_predictions'])

In [13]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 435 entries, 0 to 434
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   original_sentences    435 non-null    object
 1   original_predictions  435 non-null    int64 
 2   modified_sentences    435 non-null    object
 3   modified_predictions  435 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 13.7+ KB


#### 5. Creation of adversarial Dataset

In [14]:
advds = ds[ds.original_predictions != ds.modified_predictions]

In [15]:
advds.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56 entries, 3 to 426
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   original_sentences    56 non-null     object
 1   original_predictions  56 non-null     int64 
 2   modified_sentences    56 non-null     object
 3   modified_predictions  56 non-null     int64 
dtypes: int64(2), object(2)
memory usage: 2.2+ KB


In [16]:
 advds.to_json(r'data/adversarial_dataset_punctuation.json', orient='records')